In [1]:
!pip3 install wandb

In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler
import wandb

In [3]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projects/Covid19-3/worldometer_coronavirus_daily_data.csv')

In [4]:
data.head()

,date,country,cumulative_total_cases,daily_new_cases,active_cases,cumulative_total_deaths,daily_new_deaths
0,2020-2-15,Afghanistan,0.0,NaN,0.0,0.0,NaN
1,2020-2-16,Afghanistan,0.0,NaN,0.0,0.0,NaN
2,2020-2-17,Afghanistan,0.0,NaN,0.0,0.0,NaN
3,2020-2-18,Afghanistan,0.0,NaN,0.0,0.0,NaN
4,2020-2-19,Afghanistan,0.0,NaN,0.0,0.0,NaN


In [5]:
data.tail()

,date,country,cumulative_total_cases,daily_new_cases,active_cases,cumulative_total_deaths,daily_new_deaths
85648,2021-3-07,Zimbabwe,36271.0,11.0,952.0,1485.0,0.0
85649,2021-3-08,Zimbabwe,36289.0,18.0,930.0,1487.0,2.0
85650,2021-3-09,Zimbabwe,36321.0,32.0,913.0,1489.0,2.0
85651,2021-3-10,Zimbabwe,36341.0,20.0,899.0,1489.0,0.0
85652,2021-3-11,Zimbabwe,36377.0,36.0,913.0,1492.0,3.0


In [6]:
data.drop(['cumulative_total_cases','cumulative_total_deaths','active_cases'],axis=1,inplace=True)

In [7]:
data.head()

,date,country,daily_new_cases,daily_new_deaths
0,2020-2-15,Afghanistan,NaN,NaN
1,2020-2-16,Afghanistan,NaN,NaN
2,2020-2-17,Afghanistan,NaN,NaN
3,2020-2-18,Afghanistan,NaN,NaN
4,2020-2-19,Afghanistan,NaN,NaN


In [8]:
data.tail()

,date,country,daily_new_cases,daily_new_deaths
85648,2021-3-07,Zimbabwe,11.0,0.0
85649,2021-3-08,Zimbabwe,18.0,2.0
85650,2021-3-09,Zimbabwe,32.0,2.0
85651,2021-3-10,Zimbabwe,20.0,0.0
85652,2021-3-11,Zimbabwe,36.0,3.0


In [9]:
data.dropna(inplace=True)

In [10]:
data.head()

,date,country,daily_new_cases,daily_new_deaths
10,2020-2-25,Afghanistan,0.0,0.0
11,2020-2-26,Afghanistan,0.0,0.0
12,2020-2-27,Afghanistan,0.0,0.0
13,2020-2-28,Afghanistan,0.0,0.0
14,2020-2-29,Afghanistan,0.0,0.0


In [11]:
len(data)

67079

In [12]:
import re
dates = []
for date in data['date']:
  dates.append(int(re.sub('-','',str(date))))

In [13]:
dates[:1]

[2020225]

In [14]:
data['date'] = dates

In [15]:
# big_data,data = train_test_split(data,test_size=0.25)

In [16]:
X = data.drop(['daily_new_cases','daily_new_deaths'],axis=1)
y = data.drop(['date','country'],axis=1)

In [17]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)

In [18]:
X

,date,country
10,2020225,Afghanistan
11,2020226,Afghanistan
12,2020227,Afghanistan
13,2020228,Afghanistan
14,2020229,Afghanistan
...,...,...
85648,2021307,Zimbabwe
85649,2021308,Zimbabwe
85650,2021309,Zimbabwe
85651,2021310,Zimbabwe


In [19]:
y

,daily_new_cases,daily_new_deaths
10,0.0,0.0
11,0.0,0.0
12,0.0,0.0
13,0.0,0.0
14,0.0,0.0
...,...,...
85648,11.0,0.0
85649,18.0,2.0
85650,32.0,2.0
85651,20.0,0.0


In [20]:
ct = make_column_transformer(
    (MinMaxScaler(),['date']),
    (OneHotEncoder(),['country'])
)
ct.fit(X)
X_train = ct.transform(X_train)
X_test = ct.transform(X_test)

In [21]:
y_train = tf.constant(y_train)
y_test = tf.constant(y_test)

In [22]:
class Experiment:
  def experiment(self,amount_of_layers:int,epochs:int,lr:int,optimizer_,neurons_per_layer:int,activation_types:list,metrics_:list,name:str,project_name:str):
    wandb.init(project=project_name,name=name)
    model = tf.keras.Sequential()
    if int(amount_of_layers) != len(activation_types):
      activation_types_new = []
      for _ in range(amount_of_layers):
        activation_types_new.append(activation_types[0])
    for activation_type in activation_types:
      model.add(tf.keras.layers.Dense(neurons_per_layer,activation=activation_type))
    model.add(tf.keras.layers.Dense(2))
    model.compile(loss=tf.keras.losses.mae,metrics=metrics_,optimizer=optimizer_(lr=lr)) # change
    model.fit(X_train,tf.constant(y_train),validation_data=(X_test,tf.constant(y_test)),epochs=epochs,callbacks=[WandbCallback()]) # callbacks # change
    return model

In [23]:
from wandb.keras import WandbCallback

In [24]:
len(y_train)

50309

In [25]:
len(y_test)

16770

In [26]:
# Layers = 1,2,3,4,5,10,15,20,25
# Epochs = 125,250,500,750,1000
# learning rate = 1.0,0.1,0.01,0.001
# total neurons = 25000,50000,75000,100000,125000,250000
# activation type = linear Swish ReLU Tanh

In [27]:
E = Experiment()
model = E.experiment(amount_of_layers=1,epochs=100,lr=0.1,optimizer_=tf.keras.optimizers.Adam,neurons_per_layer=1,activation_types=['swish'],metrics_=['mae','mse'],name='test',project_name='Covid19-3')
y_preds = model.predict(X_test)

wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)


Epoch 1/100
1573/1573 [==============================] - 5s 3ms/step - loss: 844.7671 - mae: 844.7671 - mse: 42938228.6598 - val_loss: 720.5106 - val_mae: 720.5106 - val_mse: 35139356.0000
Epoch 2/100
1573/1573 [==============================] - 4s 2ms/step - loss: 734.0538 - mae: 734.0538 - mse: 38810265.7128 - val_loss: 670.5538 - val_mae: 670.5538 - val_mse: 29022804.0000
Epoch 3/100
1573/1573 [==============================] - 4s 2ms/step - loss: 618.8180 - mae: 618.8180 - mse: 24399321.1648 - val_loss: 643.8456 - val_mae: 643.8456 - val_mse: 25230248.0000
Epoch 4/100
1573/1573 [==============================] - 4s 2ms/step - loss: 625.2990 - mae: 625.2990 - mse: 23793357.3872 - val_loss: 635.5453 - val_mae: 635.5453 - val_mse: 23400768.0000
Epoch 5/100
1573/1573 [==============================] - 4s 2ms/step - loss: 623.0515 - mae: 623.0515 - mse: 23545153.7268 - val_loss: 632.5223 - val_mae: 632.5223 - val_mse: 22531496.0000
Epoch 6/100
1573/1573 [==============================] 

In [28]:
y_preds

array([[2.4881157e+01, 5.3415742e-02],
       [2.4881147e+01, 5.3415548e-02],
       [5.0293242e+04, 9.9982788e+02],
       ...,
       [2.4881151e+01, 5.3415634e-02],
       [3.8608875e+03, 7.6346764e+01],
       [1.7981568e+04, 3.5718936e+02]], dtype=float32)

In [29]:
y_test

<tf.Tensor: shape=(16770, 2), dtype=float64, numpy=
array([[0.0000e+00, 0.0000e+00],
       [1.4600e+02, 3.0000e+00],
       [2.1365e+04, 4.2300e+02],
       ...,
       [1.7400e+02, 2.0000e+00],
       [1.3571e+04, 1.3900e+02],
       [1.5277e+04, 1.5200e+02]])>

In [30]:
y_test[0]

<tf.Tensor: shape=(2,), dtype=float64, numpy=array([0., 0.])>

In [31]:
y_preds[0]

array([24.881157  ,  0.05341574], dtype=float32)

In [33]:
import time
for y_pred,y_t in zip(y_preds,y_test.numpy()):
  print(y_pred)
  print(y_t)
  print('-')

Streaming output truncated to the last 5000 lines.
[0. 0.]
-
[452.66107   8.56142]
[59.  0.]
-
[5704.7686   113.01923]
[15161.   380.]
-
[24.881159    0.05341578]
[0. 0.]
-
[24.880238    0.05339745]
[0. 0.]
-
[508.10944    9.664219]
[787.   3.]
-
[24.881159    0.05341578]
[7. 0.]
-
[24.881159    0.05341578]
[3. 0.]
-
[3287.335     64.93952]
[5996.  123.]
-
[24.881159    0.05341576]
[11.  0.]
-
[ 0.02277374 -0.44098625]
[0. 0.]
-
[4281.432     84.71086]
[0. 0.]
-
[452.7027     8.562248]
[368.  72.]
-
[8848.418    175.54248]
[302.   1.]
-
[24.881157    0.05341574]
[3. 0.]
-
[24.881159    0.05341578]
[0. 0.]
-
[24.881153    0.05341566]
[112.  24.]
-
[2567.066     50.61428]
[6597.   79.]
-
[24.881159    0.05341578]
[43.  0.]
-
[24.881159    0.05341577]
[71.  4.]
-
[294.21616     5.4101505]
[89.  0.]
-
[780.1149    15.074066]
[1072.   62.]
-
[24.880774    0.05340812]
[37.  1.]
-
[1941.7892    38.17831]
[1268.   22.]
-
[24.881159    0.05341578]
[4. 0.]
-
[23.928106    0.03446077]
[0. 0.]
-
[

In [34]:
# Layers = 1,2,3,4,5,10,15,20,25
# Epochs = 125,250,500,750,1000
# learning rate = 1.0,0.1,0.01,0.001
# total neurons = 25000,50000,75000,100000,125000,250000
# activation type = linear Swish ReLU Tanh

In [35]:
E = Experiment()

In [36]:
# I couldnt save the experiments

In [38]:
E.experiment(amount_of_layers=2,epochs=25,lr=0.1,optimizer_=tf.keras.optimizers.Adam,neurons_per_layer=25000,activation_types=['linear'],metrics_=['mae','mse'],project_name='Covid19-3',name='Big-Dog-Model')

epoch,14
loss,898.43646
mae,898.43646
mse,24046798.0
val_loss,915.7724
val_mae,915.7724
val_mse,22369226.0
_runtime,670
_timestamp,1615884407
_step,14
best_val_loss,883.56702


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▄▄▃▂▃▁▃▂▂▃▂▂▂▂
mae,█▄▄▃▂▃▁▃▂▂▃▂▂▂▂
mse,█▂▂▂▂▁▁▁▂▁▂▂▁▂▁
val_loss,█▃▂▁▂▅▆▁▂▂▂▂▂▃▂
val_mae,█▃▂▁▂▅▆▁▂▂▂▂▂▃▂
val_mse,▄▇▆█▁▅▄█▅▆▆▅▅▆▄
_runtime,▁▂▂▃▃▄▄▅▅▆▆▇▇██
_timestamp,▁▂▂▃▃▄▄▅▅▆▆▇▇██
_step,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█


Epoch 1/25
1573/1573 [==============================] - 46s 29ms/step - loss: 975.6309 - mae: 975.6309 - mse: 40451479.8412 - val_loss: 963.0080 - val_mae: 963.0080 - val_mse: 26548476.0000
Epoch 2/25
1573/1573 [==============================] - 46s 29ms/step - loss: 901.0863 - mae: 901.0863 - mse: 23650913.2043 - val_loss: 906.9406 - val_mae: 906.9406 - val_mse: 23736784.0000
Epoch 3/25
1573/1573 [==============================] - 45s 28ms/step - loss: 924.9448 - mae: 924.9448 - mse: 28127894.6571 - val_loss: 878.0183 - val_mae: 878.0183 - val_mse: 23855330.0000
Epoch 4/25
1573/1573 [==============================] - 44s 28ms/step - loss: 909.9980 - mae: 909.9980 - mse: 24041464.7386 - val_loss: 916.3779 - val_mae: 916.3779 - val_mse: 25660692.0000
Epoch 5/25
1573/1573 [==============================] - 45s 28ms/step - loss: 910.5893 - mae: 910.5893 - mse: 28020029.9331 - val_loss: 919.0647 - val_mae: 919.0647 - val_mse: 22379034.0000
Epoch 6/25
1573/1573 [============================